### Importing Important Library

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
from torchtext.legacy.datasets import IMDB
from torchtext.legacy.data import Field, LabelField, BucketIterator

In [3]:
from tqdm import tqdm
import random
import sys

### Version chcek

In [4]:
f'Torch CUDA Version :{torch.version.cuda}'
f'Torch Version :{torch.__version__}'
f'Python Version :{sys.version}'

'Torch CUDA Version :10.2'

'Torch Version :1.8.1'

'Python Version :3.8.10 (default, May 19 2021, 18:05:58) \n[GCC 7.3.0]'

### GPU Checker 

In [5]:
def gpu_check(seed_val = 1):
    print('The Seed is set to {}'.format(seed_val))
    if torch.cuda.is_available():
        print('Model will Run on CUDA.')
        print ("Type 'watch nvidia-smi' to monitor GPU\n")
        torch.cuda.manual_seed(seed_val)
        device = 'cuda'
    else:
        torch.manual_seed(seed_val)
        print ('Running in CPU')
        device = 'cpu'
    cuda = torch.cuda.is_available()
    return cuda,seed_val,device

In [6]:
cuda,SEED,device = gpu_check(seed_val=1234)

The Seed is set to 1234
Model will Run on CUDA.
Type 'watch nvidia-smi' to monitor GPU



In [7]:
!nvidia-smi

Thu May 27 17:48:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 960M    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P0    N/A /  N/A |    454MiB /  4046MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
def tokenize(s):
    return s.split(' ')

### Removing spacy for faster execution and loading custion tokenizer

In [9]:
TEXT = Field(tokenize = 'spacy',tokenizer_language = 'en_core_web_sm', lower = True,include_lengths = True)
# TEXT = Field(tokenize = tokenize,tokenizer_language = 'en_core_web_sm', lower = True)
LABEL = LabelField(dtype = torch.float)

### Creating the data split 

In [10]:
train_data, test_data = IMDB.splits(TEXT, LABEL,root ='/home/jd/Desktop/DATASET/IMDB_data/')

In [11]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [12]:
# display lenght of test and traing data
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 17500
Number of testing examples: 25000


In [13]:
# display single example at index 0
print(vars(train_data.examples[0]))

{'text': ['a', 'terrible', 'movie', 'as', 'everyone', 'has', 'said', '.', 'what', 'made', 'me', 'laugh', 'was', 'the', 'cameo', 'appearance', 'by', 'scott', 'mcnealy', ',', 'giving', 'an', 'award', 'to', 'one', 'of', 'the', 'murdered', 'programmers', 'in', 'front', 'of', 'a', 'wall', 'of', 'sun', 'logos', '.', 'mcnealy', 'is', 'the', 'ceo', 'of', 'sun', 'microsystem', ',', 'a', 'company', 'that', 'practically', 'defines', 'itself', 'by', 'its', 'hatred', 'of', 'microsoft', '.', 'they', 'have', 'been', 'instrumental', 'in', 'filing', 'antitrust', 'complaints', 'against', 'microsoft', '.', 'so', ',', 'were', 'they', 'silly', 'enough', 'to', 'think', 'this', 'bad', 'movie', 'would', 'add', 'fuel', 'to', 'that', 'fire?<br', '/><br', '/>there', "'s", 'no', 'public', 'record', 'i', 'see', 'of', 'sun', "'s", 'involvement', ',', 'but', 'clearly', 'the', 'makers', 'of', 'this', 'movie', 'know', 'scott', 'mcnealy', '.', 'an', 'interesting', 'mystery', '.'], 'label': 'neg'}


### Visualizing the data

In [14]:
line = ''
filenumber = 11

for i in train_data.examples[filenumber].text:
    line += i + ' '
line

train_data.examples[filenumber].label

"wow . one of the greatest movies i have ever - ever seen.<br /><br />absolutely loved it ! before the opening credits were done i was glued to the screen.<br /><br />it 's a sci - fi thriller - and edge of your seat whodunnit . incredible.<br /><br />i wish'd it would never end.<br /><br />lucy liu is a throwaway role . anyone could have played it . the lead actor , jeremy northram was the perfect geeky guy . < br /><br />this movie appeals to me who loved war games , sneakers , and track down.<br /><br />incredible!<br /><br />8 - 22 - 06 . walt d in lv "

'pos'

###  Build vocabulary for source and target from training data
 

In [15]:
# Build vocabulary for source and target from training data

TEXT.build_vocab(train_data, max_size=25_000)
LABEL.build_vocab(train_data)


In [16]:

# print(vars(TEXT.vocab))
print(f"Unique tokens in source vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in TRG vocabulary: {len(LABEL.vocab)}")

Unique tokens in source vocabulary: 25002
Unique tokens in TRG vocabulary: 2


### train and test iteartor

In [17]:
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    device = device)

In [18]:
for batch in train_iterator:
    text, text_length = batch.text
#     logits = model(text, text_length)
    break

In [19]:
text_length
text.shape

tensor([134, 134, 134, 134, 134, 134, 134, 134, 134, 133, 133, 133, 133, 133,
        133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133,
        133, 133, 132, 132], device='cuda:0')

torch.Size([134, 32])

### Creating Model

In [20]:
# Model class
class Model(nn.Module):
    def __init__(self, input_dim, output_dim,emb_dim, hidden_dim, n_layers, dropout):
        # input_dim <--- vocabulary size
        # output_dim <--- len ([positive, negative]) == 2 
        # emb_dim <--- embedding dimension of embedding matrix

        super(Model, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)

        self.fc1 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, src,Len):
        # shape: [source_len, batch_size]
        embedded = self.dropout(self.embedding(src)) # shape: [src_len, batch_size, embed_dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, Len.to('cpu'))
        output, (hidden, cell) = self.rnn(packed) 
        # output shape -> [batch, hidden_dim]
        # hiddden shape -> [n_layers, batch, hidden_dim]
        # cell shape -> [n_layers, batch, hidden_dim]
        output = self.fc1(hidden)
#         output = self.fc2(self.relu(output))
        return output.squeeze(0)

In [21]:
#initializing variables and hyper parameters
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(LABEL.vocab)

EMBEDDING_DIM = 128
HIDDEN_DIM = 256

N_LAYERS = 1
DROPOUT = 0.6

# initializing our model
model = Model(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(device)



/home/jd/anaconda3/envs/eva5/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [22]:
train_loss = []
train_accuracy = []

In [23]:
# loop and train our model
optimizer = optim.Adam(model.parameters(), lr=3e-3)

# defining learnig rate scheduler (optional)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

criterion = nn.CrossEntropyLoss()


### Model training function

In [24]:


def train(EPOCH,model, iterator, optimizer=optimizer, criterion=criterion, clip=1,):
    model.train()
    epoch_loss = 0
    total_correct = 0
    total_count = 0
    pbar = tqdm(iterator)
    for i, batch in enumerate(pbar):
        src,data_len = batch.text
        src = src.to(device)
        trg = batch.label.to(device)
        trg = trg.long()
        optimizer.zero_grad()
        output = model(src,data_len)
        
        total_correct += torch.sum(torch.eq(output.argmax(1), trg))
        total_count+=len(trg)
        
        loss = criterion(output, trg)
        
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        pbar.set_description(desc= f'Epoch {EPOCH} Train data Batch No : {i} Loss : {loss.item():.3f} Accuracy : {total_correct/total_count * 100 :.2f}% ' )
    
    train_accuracy.append(total_correct/total_count)
    mean_loss = epoch_loss / len(iterator)
    train_loss.append(mean_loss)
    
    scheduler.step(mean_loss)
    

### Model Validation function

In [25]:
def evaluate(EPOCH,model, iterator, criterion,typ_loader):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    pbar  = tqdm(iterator)
    with torch.no_grad():
        
        for i,batch in enumerate(pbar):
            src,data_len = batch.text
            src = src.to(device)
            trg = batch.label.to(device)
            trg = trg.long()
            predictions = model(src,data_len)
            
            loss = criterion(predictions, trg)
            
            acc = binary_accuracy(predictions, trg)

            epoch_loss += loss.item()
            epoch_acc += acc
            pbar.set_description(desc= f'Epoch {EPOCH} {typ_loader} Batch No : {i} Loss : {loss.item():.3f} Accuracy : {epoch_acc / len(iterator)* 100 :.2f}% ' )

In [26]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds.argmax(1) == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc.item()

In [27]:

total_epoch = 30
for epoch in range(total_epoch):
    result = train(epoch,model=model, iterator=train_iterator)
    evaluate(epoch,model,valid_iterator,criterion,'Valid data')
    evaluate(epoch,model,test_iterator,criterion,'Test data')
    

Epoch 0 Train data Batch No : 546 Loss : 0.613 Accuracy : 56.30% : 100%|██████████| 547/547 [00:30<00:00, 18.19it/s]
Epoch 0 Valid data Batch No : 234 Loss : 0.604 Accuracy : 66.63% : 100%|██████████| 235/235 [00:05<00:00, 43.10it/s]
Epoch 0 Test data Batch No : 781 Loss : 0.556 Accuracy : 66.55% : 100%|██████████| 782/782 [00:17<00:00, 44.03it/s]
Epoch 1 Train data Batch No : 546 Loss : 0.490 Accuracy : 70.71% : 100%|██████████| 547/547 [00:30<00:00, 17.93it/s]
Epoch 1 Valid data Batch No : 234 Loss : 0.372 Accuracy : 80.70% : 100%|██████████| 235/235 [00:05<00:00, 44.58it/s]
Epoch 1 Test data Batch No : 781 Loss : 0.392 Accuracy : 80.68% : 100%|██████████| 782/782 [00:17<00:00, 44.02it/s]
Epoch 2 Train data Batch No : 546 Loss : 0.377 Accuracy : 81.60% : 100%|██████████| 547/547 [00:32<00:00, 16.61it/s]
Epoch 2 Valid data Batch No : 234 Loss : 0.086 Accuracy : 83.36% : 100%|██████████| 235/235 [00:06<00:00, 38.94it/s]
Epoch 2 Test data Batch No : 781 Loss : 0.616 Accuracy : 82.59% : 

### function to experiment movie review sentences

In [30]:

import spacy
sp = spacy.load('en_core_web_sm')


def predict(sentence):
    if type(sentence) == str:
        tokanized_sentence = [word.text for word in sp.tokenizer(sentence)]
    else:
        tokanized_sentence = sentence


    input_data = [TEXT.vocab.stoi[word.lower()] for word in tokanized_sentence]
    input_data = torch.tensor(input_data, dtype=torch.int64).unsqueeze(1).to(device)
    

    model.eval()
    output = model(input_data)
    # print(output)
    predict = output.argmax(1)
    predict = predict.squeeze(0)
    print(output)

    if predict>0:
        return "---->> Positive Review"
    else:
        return '---->> Negative Review'

In [31]:
predict('Very bad') # predict funciton will predict if this is positive or negative review.

AttributeError: 'int' object has no attribute 'to'

In [ ]:
predict('Very good') # predict funciton will predict if this is positive or negative review.

In [ ]:
predict('i recommend to watch the movie once. It is mindblowing') # predict funciton will predict if this is positive or negative review.